In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [5]:
from turtle import Turtle, Screen
import random
from math import cos, radians


class Schooler(Turtle):
    swarm = []

    def __init__(self):
        Turtle.__init__(self)
        self.up()
        self.setheading(random.randrange(360))
        self.setpos(random.randrange(-200,200),random.randrange(-200,200))
        self.down()
        self.newHead = None
        Schooler.swarm.append(self)

    def getNewHeading(self):
        minangle = 999
        for other in Schooler.swarm:
            if self != other:
                head = self.towards(other) - self.heading()
                if cos(radians(head)) > 0:
                    if head < minangle:
                        minangle = head
        if minangle == 999:
            self.newHead = self.heading()
        else:
            self.newHead = minangle+self.heading()

    def inFrontOf(self,other):
        head = self.towards(other) - self.heading()
        if cos(radians(head)) > 0:
            return True
        return False

    def setHeadingAndMove(self):
        self.setheading(self.newHead)
        self.newHead = None
        self.forward(10)

class FocalFish(Schooler):
    repulse = 10
    align = 50
    attract = 600

    def getNewHeading(self):
        repulsion = []
        alignment = []
        attraction = []

        for other in Schooler.swarm:
            if self != other:
                dist = self.distance(other)
                if dist <= self.repulse:
                    repulsion.append(other)
                elif dist <= self.align:
                    alignment.append(other)
                elif dist <= self.attract:
                    attraction.append(other)

        self.newHead = self.heading()
        if repulsion:
            x = 0
            y = 0
            for o in repulsion:
                x = x + o.xcor()
                y = y + o.ycor()

            self.newHead = self.towards(x/len(repulsion),y/len(repulsion)) + 180

        elif alignment:
            hs = self.heading()
            for other in alignment:
                hs = hs + other.heading()
            self.newHead = hs // (len(alignment)+1)

        elif attraction:
            x = 0
            y = 0
            for o in attraction:
                x = x + o.xcor()
                y = y + o.ycor()
            self.newHead = self.towards(x/len(attraction),y/len(attraction))




def main():
    swarmSize = 50
    t = Turtle()
    win = Screen()
    win.setworldcoordinates(-600,-600,600,600)
    t.speed(10)
    t.hideturtle()
    t.tracer(15)

    for i in range(swarmSize):
        FocalFish()

    for turn in range(300):
        for schooler in Schooler.swarm:
            schooler.getNewHeading()

        for schooler in Schooler.swarm:
            schooler.setHeadingAndMove()

    win.exitonclick()


main()


TclError: invalid command name ".4483265456"

## Jäger und Schwarm

In [10]:
import numpy
import time
import sys
import math
 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
 
 
#################
# Runtime parameters
NumberOfBoids       = 100
NumberOfPredators   = 2
Dimensions          = 3
MinSeparation       = 7
framespersecond     = 24
timesteps           = 240 #Should be divisible by framespersecond
 
# Predator settings
PredatorRadius      = 25
PredatorSight       = 10
WeightCenterOfMassP = 0.00
WeightAttackBoid    = 0 #2
WeightRandomP       = 0.5 #1
WeightKnot          = 2
 
# Boids behavior - weight on rules
WeightCenterOfMass  = 0.03
WeightSeparation    = 1
WeightAlignment     = 0.125
WeightRandom        = 0.0
WeightAvoidPredator = 1
WeightCenter        = 0.02 #0.03
 
MaxVelocityP        = 1
MaxVelocity         = 1
center              = 50*numpy.ones(Dimensions)
###########
 
class boid:
    """This class defines the boid and the rules of behavoir"""
    def __init__(self, dimension, index):
        self.index  = index
        self.dim    = dimension
        self.pos    = numpy.random.uniform(0,100,self.dim)
        self.vel    = numpy.zeros(self.dim)
        self.size   = numpy.random.rand(1)
 
    def limitvelocity(self,maxvel):
        """Limiting the speed to avoid unphysical jerks in motion"""
        if numpy.linalg.norm(self.vel) > maxvel:
            self.vel = (self.vel/numpy.linalg.norm(self.vel))*maxvel
 
def normalize(vector):
    """Normalizes a vector"""
    vector = vector/numpy.linalg.norm(vector)
    return vector
 
def centerofpos(flock):
    """Calculates the 'center of mass' for the flock of boids"""
    com = numpy.zeros(Dimensions)   
    for dim in range(Dimensions):
        for boid in flock:
            com[dim] = com[dim] + boid.pos[dim]
        com[dim] = com[dim]/len(flock)
    return com
 
def centerofvel(flock):
    """Calculates the mean velocity vector for the flock of boids"""
    cov = numpy.zeros(Dimensions)   
    for dim in range(Dimensions):
        for boid in flock:
            cov[dim] = cov[dim] + boid.vel[dim]
        cov[dim] = cov[dim]/len(flock)
    return cov
 
def plot_raw():
    """Sets up the basic plotting paramaters"""
    global ax
    ax = fig.add_subplot(111, projection='3d')
    ax.set_autoscale_on(False)
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    ax.set_xlim3d([0.0, 100.0])
    ax.set_ylim3d([0.0, 100.0])
    ax.set_zlim3d([0.0, 100.0])
 
def plot_init():
    """The initial draw"""
    plot_raw()
    com = centerofpos(flock)
    boidpos = numpy.zeros((NumberOfBoids, Dimensions))
    predpos = numpy.zeros((NumberOfPredators, Dimensions))
 
    for index,boid in enumerate(flock):
        boidpos[index]= boid.pos       
    bx = boidpos[:,0]
    by = boidpos[:,1]
    bz = boidpos[:,2] 
 
    for index,predator in enumerate(predators):
        predpos[index]= predator.pos       
    px = predpos[:,0]
    py = predpos[:,1]
    pz = predpos[:,2] 
 
def plot_update(timestep):
    """ This plot updates the plot with the new positions of the boids
    and predators"""
    fig.clf()
    plot_raw()
 
    # Execute a calculation of the next set of moves
    mainloop(timestep)
 
    # Plots the boids in space using matplotlib
    # http://matplotlib.org/examples/mplot3d/scatter3d_demo.html         
    boidpos = numpy.zeros((NumberOfBoids, Dimensions))
    predpos = numpy.zeros((NumberOfPredators, Dimensions))
 
    # Plot center of mass as a black dot
    com = centerofpos(flock)
    ax.scatter(com[0], com[1], com[2], color='black')
 
    # Plot boids as blue dots     
    for index,boid in enumerate(flock):
        boidpos[index]= boid.pos       
    bx = boidpos[:,0]
    by = boidpos[:,1]
    bz = boidpos[:,2] 
    ax.scatter(bx, by, bz, c='b')
 
    # Predators are red dots
    if len(predators) > 0:
        for index,predator in enumerate(predators):
            predpos[index]= predator.pos       
        px = predpos[:,0]
        py = predpos[:,1]
        pz = predpos[:,2] 
        ax.scatter(px, py, pz, s=100,color='red')      
 
def mainloop(timestep):
    """This is the calculation timeloop"""
    print "Timestep "+str(timestep)
 
    if len(predators) > 0:
        for predator in predators:
 
            # Predator RULE 1.  Cohesion - Steer to move towoards the center of mass
            prule1 = numpy.zeros(Dimensions)
            prule1 = (centerofpos(flock) - predator.pos)*WeightCenterOfMassP    
 
            # Predator RULE 2. (unused and now just a placeholder
            prule2 = numpy.zeros(Dimensions)
 
            # Predator RULE 3. Attack boids within range
            prule3 = numpy.zeros(Dimensions)
            for boid in flock:
                difference  = predator.pos - boid.pos
                distance    = numpy.linalg.norm(difference)
                if distance < PredatorSight:
                    prule3 = prule3 - normalize(difference)
            prule3 = prule3*WeightAttackBoid
 
            # Predator RULE 4. Move the predator around in smooth way around the center of the cube
            # http://en.wikipedia.org/wiki/Trefoil_knot
            prule4 = numpy.zeros(Dimensions)
            t = (timestep/float(timesteps))*4*math.pi + predator.index*(math.pi/4.0)
            if Dimensions == 3: 
                prule4[0] = (2.0+math.cos(3.0*t))*math.cos(2.0*t)
                prule4[1] = (2.0+math.cos(3.0*t))*math.sin(2.0*t)
                prule4[2] = math.sin(3*t)
            else:
                prule4 = numpy.zeros(Dimensions)
            prule4 = prule4 * WeightKnot
 
            # Move the predator
            predator.vel = prule1 + prule2 + prule3 + prule4
            predator.limitvelocity(MaxVelocityP)
            predator.pos = predator.pos + predator.vel
 
    for boid in flock:
 
        # Boid RULE 1. Cohesion - Steer to move towoards the center of mass
        rule1 = numpy.zeros(Dimensions)
        rule1 = (centerofpos(flock) - boid.pos)*WeightCenterOfMass
 
        # Boid RULE RULE 2. Separation - steer to avoid crowding local flockmates
        rule2 = numpy.zeros(Dimensions)
        for boid2 in flock:
            difference  = boid2.pos - boid.pos
            distance    = numpy.linalg.norm(difference)
            if distance < MinSeparation and boid2 != boid:
                rule2 = rule2 - normalize(difference)/distance
        rule2 = rule2*WeightSeparation 
 
        # Boid RULE 3. Alignment - Steer towards the average heading of local flockmates
        rule3 = numpy.zeros(Dimensions)
        rule3 = (centerofvel(flock) - boid.vel)*WeightAlignment
 
 
        # The following rules are custom rules just added for fun. 
        # Boid RULE 4. Try to move towards the center of the grid
        rule4 = (center - boid.pos)*WeightCenter
 
        # Boid RULE 5. Add some randomness
        rule5 = numpy.random.uniform(-1,1,Dimensions)*WeightRandom
 
        # Boid RULE 6. Avoid the predator
        if len(predators) > 0:
            rule6 = numpy.zeros(Dimensions)
            for predator in predators:
                difference  = predator.pos - boid.pos
                distance    = numpy.linalg.norm(difference)
                if distance < PredatorRadius:
                    rule6 = (rule6 - difference)*WeightAvoidPredator
        else:
            rule6 = numpy.zeros(Dimensions)
 
        # Move the boids
        boid.vel = rule1 + rule2 + rule3 + rule4 + rule5 + rule6
        boid.limitvelocity(MaxVelocity)
        boid.pos = boid.pos + boid.vel
 
 
 
#Generate the the flock of boids
flock           = [boid(Dimensions,count) for count in range(NumberOfBoids)]
 
if NumberOfPredators == 0:
    predators   = []    
else:
    predators   = [boid(Dimensions,count) for count in range(NumberOfPredators)]    
    # HACK - centered starting positions
    for predator in predators:
        predator.pos = numpy.random.uniform(45,55,Dimensions)
 
# Define the figure:
fig = plt.figure()
plot_raw()
 
anim = animation.FuncAnimation(fig, plot_update, init_func=plot_init,frames=timesteps, interval=20, blit=True)
anim.save('basic_animation.mp4', fps=framespersecond, codec='mpeg4')


Timestep 0
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Timestep 13
Timestep 14
Timestep 15
Timestep 16
Timestep 17
Timestep 18
Timestep 19
Timestep 20
Timestep 21
Timestep 22
Timestep 23
Timestep 24
Timestep 25
Timestep 26
Timestep 27
Timestep 28
Timestep 29
Timestep 30
Timestep 31
Timestep 32
Timestep 33
Timestep 34
Timestep 35
Timestep 36
Timestep 37
Timestep 38
Timestep 39
Timestep 40
Timestep 41
Timestep 42
Timestep 43
Timestep 44
Timestep 45
Timestep 46
Timestep 47
Timestep 48
Timestep 49
Timestep 50
Timestep 51
Timestep 52
Timestep 53
Timestep 54
Timestep 55
Timestep 56
Timestep 57
Timestep 58
Timestep 59
Timestep 60
Timestep 61
Timestep 62
Timestep 63
Timestep 64
Timestep 65
Timestep 66
Timestep 67
Timestep 68
Timestep 69
Timestep 70
Timestep 71
Timestep 72
Timestep 73
Timestep 74
Timestep 75
Timestep 76
Timestep 77
Timestep 78
Timestep 79
Timestep 80
Timestep 81
Timestep 82
Timestep 83
Ti